**In this tutorial, We will use an ML Technique to detect fraudulent transaction using credit cards**

## Importing Dependencies and modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import sklearn

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
credit_card_data=pd.read_csv('/content/drive/MyDrive/ML Projects SkillSync Internship/creditcard.csv')

In [ ]:
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


**Now similary explore the last 3 samples**

In [ ]:
credit_card_data.tail(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,...,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0


In [ ]:
credit_card_data.shape

(284807, 31)

**So, the above data shape denotes that it contains total 284807 number of samples with 31 columns; 30 columns are the attributes for data preprocessing and training purpose whil the 31st column is the class column**

Next, we'll explore the class column

In [ ]:
credit_card_data['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

the **Class** columns states that it has two values: 0 and 1. Here,

**0= normal transaction**

**1= fraudulent transaction**

In [ ]:
credit_card_data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [ ]:
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [ ]:
#method 2 to check null values:

credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

So, no null values in the dataset

# Exploratory data analysis

In [ ]:
sns.countplot(credit_card_data['Class'])

# Balancing the data

ceating a separate dataframes for 2 classes and then join the 2 class distributions after taking random values of fraud class from its enormous quantity

In [ ]:
#Class quantity:
credit_card_data['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [ ]:
#separating data for analysis:
genuine= credit_card_data[credit_card_data.Class == 0]
fraud= credit_card_data[credit_card_data.Class == 1]

print the shape of genuine and fraud dataframes

In [ ]:
print(genuine.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


now we'll take 492 genuine case to match with 492 fraud case for a more class balanced dataset for model training..

In [ ]:
genuine_sample= genuine.sample(n=492)

Concatenating 2 dataframes: fraud with genuine_sample

In [ ]:
new_data=pd.concat([genuine_sample, fraud],axis=0) # axis=0 means to add the dataframes row wise

In [ ]:
new_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
9875,14553.0,-0.735775,0.500883,-0.925039,-0.031661,3.514750,2.810834,-0.042941,0.662573,0.435883,...,-0.208434,-0.448643,0.147817,0.857727,-0.140164,0.396795,-0.024560,0.199661,0.76,0
18332,29397.0,-0.604250,1.077053,0.337986,-0.233145,0.262190,-0.317004,0.572632,0.410474,-0.942575,...,0.116016,0.108117,-0.132072,-0.487646,-0.237799,0.291921,-0.049107,0.067246,30.59,0
45832,42514.0,0.900742,-0.239704,0.103165,1.118123,-0.112731,0.144652,0.091332,0.102381,-0.188181,...,0.206359,0.300798,-0.249043,-0.299549,0.544969,-0.261513,0.000488,0.028356,144.00,0
262576,160546.0,0.007689,0.399050,-1.586777,-0.947576,2.219490,3.955335,-0.385618,1.415638,0.654690,...,-0.209914,-0.505827,0.221847,0.688915,-0.492090,-0.540583,-0.157250,-0.080911,18.90,0
140850,83977.0,-1.158716,1.235777,1.577164,0.975445,-0.577706,0.503425,0.163276,0.157500,0.307721,...,0.308393,1.094642,-0.128225,0.146935,-0.387698,-0.283124,-0.696933,-0.120712,44.95,0


In [ ]:
new_data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1
281674,170348.0,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,...,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53,1


So, the data has been taken in random manner and concatenated successfully...

In [ ]:
new_data['Class'].value_counts()

0    492
1    492
Name: Class, dtype: int64

the data is now class balanced with each class having equal number of samples--> 492

Splitting data into features and target

In [ ]:
X= new_data.drop(columns='Class', axis=1)
Y= new_data['Class']

In [ ]:
X

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
9875,14553.0,-0.735775,0.500883,-0.925039,-0.031661,3.514750,2.810834,-0.042941,0.662573,0.435883,...,0.170307,-0.208434,-0.448643,0.147817,0.857727,-0.140164,0.396795,-0.024560,0.199661,0.76
18332,29397.0,-0.604250,1.077053,0.337986,-0.233145,0.262190,-0.317004,0.572632,0.410474,-0.942575,...,-0.127951,0.116016,0.108117,-0.132072,-0.487646,-0.237799,0.291921,-0.049107,0.067246,30.59
45832,42514.0,0.900742,-0.239704,0.103165,1.118123,-0.112731,0.144652,0.091332,0.102381,-0.188181,...,0.098020,0.206359,0.300798,-0.249043,-0.299549,0.544969,-0.261513,0.000488,0.028356,144.00
262576,160546.0,0.007689,0.399050,-1.586777,-0.947576,2.219490,3.955335,-0.385618,1.415638,0.654690,...,-0.282672,-0.209914,-0.505827,0.221847,0.688915,-0.492090,-0.540583,-0.157250,-0.080911,18.90
140850,83977.0,-1.158716,1.235777,1.577164,0.975445,-0.577706,0.503425,0.163276,0.157500,0.307721,...,-0.162382,0.308393,1.094642,-0.128225,0.146935,-0.387698,-0.283124,-0.696933,-0.120712,44.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,1.252967,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.226138,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.247968,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.306271,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00


In [ ]:
Y

9875      0
18332     0
45832     0
262576    0
140850    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64

# splitting data into training and testing data

In [ ]:
#import train test split from sklearn:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test= train_test_split(X,Y, test_size=.2, stratify=Y, random_state=2)

In [ ]:
print (X.shape, X_train.shape, X_test.shape) # to see the 80-20 ratio for the training/testing purpose

(984, 30) (787, 30) (197, 30)


#Machine Learning model training: Logistic Regression





In [ ]:
#import the model:
from sklearn.linear_model import LogisticRegression

In [ ]:
#instantiate model object:
model= LogisticRegression()

fit the training data to train the regression model:

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

# MOdel Evaluation

##Accuracy score

In [ ]:
from sklearn.metrics import accuracy_score

###1. Accuracy on training data

In [ ]:
X_train_prediction= model.predict(X_train)

In [ ]:
training_accuracy= accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('training accuracy:', training_accuracy)

training accuracy: 0.9504447268106735


##2. Accuracy on testing data

In [ ]:
X_test_prediction=model.predict(X_test)

In [ ]:
test_accuracy=accuracy_score(X_test_prediction, Y_test)

In [ ]:
print (f'testing accuracy {test_accuracy}')

testing accuracy 0.934010152284264


training= 95, test= 93-----> so overfitting